In [1]:
import requests
from requests.auth import HTTPBasicAuth
import json
import base64
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")


## JIRA Spring

In [2]:
auth=HTTPBasicAuth("ShengL", "Ls2010506-s")

#### Board

In [4]:
board_url = 'https://jira.spring.io/rest/agile/1.0/board' 

headers = {
   "Accept": "application/json"
}

board = requests.request("GET",board_url,headers=headers,auth=auth).json()

board_df = pd.json_normalize(board['values'])

board_df.to_csv('jiraSpring_board.csv')  
board_id = board_df['id']
board_df.head()

,id,self,name,type
0,50,https://jira.spring.io/rest/agile/1.0/board/50,ABCTesting,scrum
1,27,https://jira.spring.io/rest/agile/1.0/board/27,alg engine,scrum
2,3,https://jira.spring.io/rest/agile/1.0/board/3,AMQP,scrum
3,45,https://jira.spring.io/rest/agile/1.0/board/45,Asmum,scrum
4,61,https://jira.spring.io/rest/agile/1.0/board/61,azerty,scrum


#### Sprint

In [5]:
headers = {
   "Accept": "application/json"
}

payload_sprint = {
'maxResults':200
}

list1 = []
for i in board_id:
   sprint_url = 'https://jira.spring.io/rest/agile/1.0/board/' + f'{i}' + '/sprint'
   sprint = requests.request("GET",sprint_url ,headers=headers,auth=auth, params=payload_sprint).json()
   try:
      list1.append(sprint['values'])
   except:
      pass

In [6]:
sprint_df = pd.json_normalize(list1)
sprint_result = pd.DataFrame()
sprint_result.head()

""


In [7]:
for i in sprint_df.index:
   sprint_df_1 = sprint_df.iloc[i].apply(pd.Series)
   sprint_df_1['board_id'] = board_df['id'][i]
   frame = [sprint_result, sprint_df_1]
   sprint_result = pd.concat(frame)

In [8]:
sprint_result = sprint_result[sprint_result['id'].notna()]
sprint_result

,board_id,id,self,state,name,startDate,endDate,completeDate,activatedDate,originBoardId,goal
0,27,14.0,https://jira.spring.io/rest/agile/1.0/sprint/14,closed,Babbage - RC1 - Stage Two,2013-07-15T13:06:47.325Z,2013-08-05T13:06:00.000Z,2013-08-05T16:06:42.063Z,2013-07-15T13:06:47.325Z,NaN,NaN
1,27,17.0,https://jira.spring.io/rest/agile/1.0/sprint/17,closed,The Road To SpringOne,2013-08-07T10:24:36.975Z,2013-09-06T10:24:36.975Z,2013-09-16T20:12:01.030Z,2013-08-07T10:24:36.975Z,NaN,NaN
2,27,22.0,https://jira.spring.io/rest/agile/1.0/sprint/22,closed,SpringOne Aftermath,2013-09-16T09:21:00.000Z,2013-10-07T09:21:00.000Z,2013-10-07T15:24:15.693Z,2013-09-16T09:21:00.000Z,NaN,NaN
3,27,23.0,https://jira.spring.io/rest/agile/1.0/sprint/23,closed,The Road to Codd M1,2013-10-07T15:24:56.220Z,2013-10-25T15:24:56.220Z,2013-11-18T10:14:01.639Z,2013-10-07T15:24:56.220Z,NaN,NaN
4,27,27.0,https://jira.spring.io/rest/agile/1.0/sprint/27,closed,Codd M1,2013-11-18T10:19:44.354Z,2014-01-06T10:19:00.000Z,2014-01-13T14:14:20.090Z,2013-11-18T10:19:44.354Z,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
46,55,80.0,https://jira.spring.io/rest/agile/1.0/sprint/80,closed,Sprint 47,2015-04-06T17:22:32.410Z,2015-04-25T01:22:00.000Z,2015-04-24T23:23:26.171Z,2015-04-06T17:22:32.410Z,6.0,NaN
47,55,82.0,https://jira.spring.io/rest/agile/1.0/sprint/82,closed,Sprint 48,2015-04-27T17:13:09.348Z,2015-05-09T00:13:00.000Z,2015-05-08T22:52:36.394Z,2015-04-27T17:13:09.348Z,6.0,NaN
48,55,83.0,https://jira.spring.io/rest/agile/1.0/sprint/83,closed,Sprint 49,2015-05-11T18:10:21.523Z,2015-05-22T23:10:00.000Z,2015-05-22T21:39:53.098Z,2015-05-11T18:10:21.523Z,6.0,NaN
49,55,85.0,https://jira.spring.io/rest/agile/1.0/sprint/85,closed,Sprint 50,2015-05-26T17:04:32.601Z,2015-06-09T01:04:00.000Z,2015-06-09T00:37:10.657Z,2015-05-26T17:04:32.601Z,6.0,NaN


In [9]:
sprint_result = sprint_result.reset_index()
sprint_result = sprint_result.drop(columns=['index'])

,board_id,id,self,state,name,startDate,endDate,completeDate,activatedDate,originBoardId,goal
0,27,14.0,https://jira.spring.io/rest/agile/1.0/sprint/14,closed,Babbage - RC1 - Stage Two,2013-07-15T13:06:47.325Z,2013-08-05T13:06:00.000Z,2013-08-05T16:06:42.063Z,2013-07-15T13:06:47.325Z,NaN,NaN
1,27,17.0,https://jira.spring.io/rest/agile/1.0/sprint/17,closed,The Road To SpringOne,2013-08-07T10:24:36.975Z,2013-09-06T10:24:36.975Z,2013-09-16T20:12:01.030Z,2013-08-07T10:24:36.975Z,NaN,NaN
2,27,22.0,https://jira.spring.io/rest/agile/1.0/sprint/22,closed,SpringOne Aftermath,2013-09-16T09:21:00.000Z,2013-10-07T09:21:00.000Z,2013-10-07T15:24:15.693Z,2013-09-16T09:21:00.000Z,NaN,NaN
3,27,23.0,https://jira.spring.io/rest/agile/1.0/sprint/23,closed,The Road to Codd M1,2013-10-07T15:24:56.220Z,2013-10-25T15:24:56.220Z,2013-11-18T10:14:01.639Z,2013-10-07T15:24:56.220Z,NaN,NaN
4,27,27.0,https://jira.spring.io/rest/agile/1.0/sprint/27,closed,Codd M1,2013-11-18T10:19:44.354Z,2014-01-06T10:19:00.000Z,2014-01-13T14:14:20.090Z,2013-11-18T10:19:44.354Z,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
815,55,80.0,https://jira.spring.io/rest/agile/1.0/sprint/80,closed,Sprint 47,2015-04-06T17:22:32.410Z,2015-04-25T01:22:00.000Z,2015-04-24T23:23:26.171Z,2015-04-06T17:22:32.410Z,6.0,NaN
816,55,82.0,https://jira.spring.io/rest/agile/1.0/sprint/82,closed,Sprint 48,2015-04-27T17:13:09.348Z,2015-05-09T00:13:00.000Z,2015-05-08T22:52:36.394Z,2015-04-27T17:13:09.348Z,6.0,NaN
817,55,83.0,https://jira.spring.io/rest/agile/1.0/sprint/83,closed,Sprint 49,2015-05-11T18:10:21.523Z,2015-05-22T23:10:00.000Z,2015-05-22T21:39:53.098Z,2015-05-11T18:10:21.523Z,6.0,NaN
818,55,85.0,https://jira.spring.io/rest/agile/1.0/sprint/85,closed,Sprint 50,2015-05-26T17:04:32.601Z,2015-06-09T01:04:00.000Z,2015-06-09T00:37:10.657Z,2015-05-26T17:04:32.601Z,6.0,NaN


In [10]:
sprint_result.to_csv('jiraSpring_sprint.csv')

#### Issues


In [11]:
headers = {
   "Accept": "application/json"
}

payload_issues = {
'maxResults':1000
}

list1 = []
for i in board_id:
   issues_url = 'https://jira.spring.io/rest/agile/1.0/board/' + f'{i}' + '/issue'
   issues = requests.request("GET",issues_url ,headers=headers,auth=auth, params=payload_issues).json()
   try:
      list1.append(issues['issues'])
   except:
      pass

In [12]:
issue_df = pd.json_normalize(list1)

In [13]:
issue_result = pd.DataFrame()

for i in issue_df.index:
   issue_df_1 = issue_df.iloc[i].apply(pd.Series)
   issue_df_1['board_id'] = board_df['id'][i]
   frame = [issue_result, issue_df_1]
   issue_result = pd.concat(frame)

issue_result = issue_result.reset_index()
issue_result = issue_result.drop(columns=['index'])

issue_result.head(5)

,expand,id,self,key,fields.issuetype.self,fields.issuetype.id,fields.issuetype.description,fields.issuetype.iconUrl,fields.issuetype.name,fields.issuetype.subtask,...,fields.customfield_10781,fields.customfield_10782,fields.customfield_10783,fields.priority,fields.creator,fields.customfield_10000,fields.customfield_10002,fields.customfield_10681,fields.customfield_10682,fields.customfield_10683
0,"operations,versionedRepresentations,editmeta,c...",40607,https://jira.spring.io/rest/agile/1.0/issue/40607,INT-2207,https://jira.spring.io/rest/api/2/issuetype/4,4,An improvement or enhancement to an existing f...,https://jira.spring.io/secure/viewavatar?size=...,Improvement,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"operations,versionedRepresentations,editmeta,c...",40701,https://jira.spring.io/rest/agile/1.0/issue/40701,INT-2228,https://jira.spring.io/rest/api/2/issuetype/4,4,An improvement or enhancement to an existing f...,https://jira.spring.io/secure/viewavatar?size=...,Improvement,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"operations,versionedRepresentations,editmeta,c...",40700,https://jira.spring.io/rest/agile/1.0/issue/40700,INT-2227,https://jira.spring.io/rest/api/2/issuetype/1,1,A problem which impairs or prevents the functi...,https://jira.spring.io/secure/viewavatar?size=...,Bug,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"operations,versionedRepresentations,editmeta,c...",40699,https://jira.spring.io/rest/agile/1.0/issue/40699,INT-2226,https://jira.spring.io/rest/api/2/issuetype/4,4,An improvement or enhancement to an existing f...,https://jira.spring.io/secure/viewavatar?size=...,Improvement,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"operations,versionedRepresentations,editmeta,c...",40693,https://jira.spring.io/rest/agile/1.0/issue/40693,INT-2225,https://jira.spring.io/rest/api/2/issuetype/3,3,A task that needs to be done.,https://jira.spring.io/secure/viewavatar?size=...,Task,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
issue_result.shape

(47000, 203)

In [15]:
issue_result.to_csv('jiraSpring_issues.csv')